In [1]:
import sapien.core as sapien
from Tools import misc, ForwardKinematics, ModelSim, NumForwardDynamicsDer, MathForwardDynamicsDer
import numpy as onp
import jax.numpy as np
from jax import jit, jacfwd, jacrev, grad, ShapedArray
from ilqr import ILQR
from datetime import datetime
from tqdm import trange
# from tqdm.notebook import trange
import matplotlib.pyplot as plt

sim = sapien.Engine()
renderer = sapien.OptifuserRenderer()
sim.set_renderer(renderer)
render_controller = sapien.OptifuserController(renderer)

Using default glsl path /home/zack/anaconda3/envs/ml/lib/python3.7/site-packages/sapien/glsl_shader/130


In [2]:
DEBUG = True

stabled = False


def create_scene(timestep, visual):
    s = sim.create_scene([0, 0, 0])
    s.add_ground(-1)
    s.set_timestep(timestep)

    loader = s.create_urdf_loader()
    loader.fix_root_link = True
    if visual:
        loader.collision_is_visual = True
        s.set_ambient_light([0.5, 0.5, 0.5])
        s.set_shadow_light([0, 1, -1], [0.5, 0.5, 0.5])

    # build
    robot = loader.load("../assets/Arm/panda.urdf")

    for joint in robot.get_joints():
        joint.set_drive_property(stiffness=0, damping=50)
    # robot.set_qpos(np.array([0, 1.7, 0, -1.5, 0, 2.5, 0.7, 0.04, 0.04]))
    p = onp.random.random(robot.dof) * 3 - 1.5
    lim = robot.get_qlimits().T
    p = onp.clip(p, lim[0], lim[1])
    robot.set_qpos(p)

    return s, robot


# renderer_timestep = 1 / 60
optim_timestep = 1 / 500
sim_timestep = 1 / 60
s0, robot = create_scene(sim_timestep, True)

render_controller.set_camera_position(-5, 0, 0)
render_controller.set_current_scene(s0)


In [3]:
def smooth_abs(x, alpha):
    return np.mean((alpha ** 2) * (np.cosh(x / alpha) - 1))


robo_pose = robot.get_root_pose()

dof = robot.dof
factor = np.array([150, 150, 150, 100, 300, 300, 300, 10, 10])
u_range = np.array([-factor, factor])
q_range = robot.get_qlimits().T
q_mean = np.mean(q_range, axis=0)
q_radius = q_range[1] - q_mean

pred_time = 0.3
horizon = int(pred_time / optim_timestep) + 1
per_iter = 2

fk = ForwardKinematics(robot)

In [45]:
@jit
def final_cost(x, alpha1=0.3, alpha2=0.5, alpha3=0.5):
    # add base pose
    qpos = x[:dof]
    qvel = x[dof:]
    pos = np.concatenate((qpos, robo_pose.p, robo_pose.q))

    cart_pos = fk.fk(pos).reshape(-1, 3)[:, :3]
    end_effector_pos = cart_pos[-3]

    target_pos = robo_pose.p + [0, 0, 1.2]
    # penalize not raising high
    term1 = smooth_abs(end_effector_pos[2] - target_pos[2], alpha1)

    # penalize high velocity
    term2 = smooth_abs(qvel / (q_radius * 2 * 3), alpha2)  # 3 seconds go full qpos span

    # penalize stuck at boundary
    term3 = smooth_abs((qpos - q_mean) / q_radius * 2, alpha3)

    return term1 * 3 + term2 * 2 + term3
